### Create external tables in Redshift DB and join inference output data


In [ ]:
!pip install psycopg2-binary

### Import Dependencies

In [ ]:
import boto3
import json
import sagemaker
import os
import psycopg2

sagemaker_session = sagemaker.Session()
s3_client = boto3.client("s3")
default_bucket = sagemaker_session.default_bucket() 
execution_role = sagemaker.get_execution_role()
secretId = "SecretForRedshiftSageMakerDemo2023"
region = boto3.Session().region_name
inference_file_location = f"s3://{default_bucket}/redshift-featurestore-blog/batch_output/"

#test_data_nolabel.csv.out

#### Retrieve redshift connection details from Secrets Manager

In [ ]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId=secretId)
database_secrets = json.loads(response['SecretString'])
username = database_secrets['username']
password = database_secrets['password']
host = database_secrets['host']
port = database_secrets['port']
engine = database_secrets['engine']
#print("jdbc:" + engine + "://" + host + ":" + str(port) + "/dev")

In [ ]:
conn=psycopg2.connect(
    dbname   = "dev", 
    host     = host, 
    port     = port, 
    user     = username, 
    password = password
)
conn.autocommit = True
cursor = conn.cursor()

#### Create external schema and table

In [ ]:
sql = """
    create external schema if not exists sagemakerdemo_external from data catalog
    database 'sm_rs_external'
    region '{}'
    iam_role '{}'
    create external database if not exists
    """.format(region, execution_role)

cursor.execute(sql)

In [ ]:
cursor.execute('drop table if exists sagemakerdemo_external.batch_output')

In [ ]:
sql = """
    create external table sagemakerdemo_external.batch_output(
    userid varchar,
    prediction numeric(3,1))
    row format delimited
    fields terminated by ','
    stored as textfile
    location  '{}'
    """.format(inference_file_location)
cursor.execute(sql)

#### Join the inference result with base table

In [ ]:
sql =  """
    select a.userid,b.prediction, a.user_latitude, a.user_longitude, a.user_smoker, a.user_drink_level 
    from sagemakerdemo.dim_user a
    join sagemakerdemo_external.batch_output b on a.userid = b.userid limit 10
    """
cursor.execute(sql)
results=cursor.fetchall()
print('\n'.join(str(x) for x in results))


#### Close the connection

In [ ]:
conn.close()